## This script will add to the table adissions the following fields:
+ treatment -> categorical
+ previous admissions -> Binary
+ Specific Comorbidities -> Binary/ Categorical
+ spin records -> Binary

In [1]:
%reset -f
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re, sqlite3, pickle, time, datetime, random, os
from dateutil.relativedelta import relativedelta

# Load Raw Data

In [2]:
t = time.time()
path  = r'/rfs/CAPAI_PhD_dflr2/dflr2/Codes_Data_2/Data/'
#file1 = path + '20210525_admissions.txt' 
#file2 = path + '20210525_eobs.txt'
file3 = path + '20210525_haem-results.txt'
#file4 = path + '20210525_icu.txt'
file5 = path + '20210525_meds.txt'
file6 = path + '20210525_micro-results.txt'
file7 = path + '20210525_oxygen.txt'
file8 = path + '20210525_prev_admissions.txt'
file9 = path + '20210525_spin.txt'
#df_admin = pd.read_csv(file1, sep='\t', lineterminator='\n')
#df_eobs  = pd.read_csv(file2, sep='\t', lineterminator='\n')
df_haemt = pd.read_csv(file3, sep='\t', lineterminator='\n')
#df_icu   = pd.read_csv(file4, sep='\t', lineterminator='\n')
df_meds  = pd.read_csv(file5, sep='\t', lineterminator='\n')
df_micro = pd.read_csv(file6, sep='\t', lineterminator='\n')
#df_oxyge = pd.read_csv(file7, sep='\t', lineterminator='\n')
df_prev  = pd.read_csv(file8, sep='\t', lineterminator='\n')
df_spin  = pd.read_csv(file9, sep='\t', lineterminator='\n')######

/local/dlr10/ipykernel_10309/3037835876.py:17: DtypeWarning: Columns (10,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_micro = pd.read_csv(file6, sep='\t', lineterminator='\n')


# 0. Load Processed parameters and data

In [3]:
##############################################################################
#######################################
### Relevant fields for pre-processing
#######################################
reference_date = datetime.datetime(1970,1,1)

##### Fields in table patients ##########
patient_field         = 'patient_id'
age_field             = 'age_at_admission\r'
sex_field             = 'sex'
ethnic_field          = 'ethnic_origin'
death_ind_field       = 'death_indicator'
death_date_field      = 'date_of_death'
death_date_code_field = 'date_code_of_death'
mort_in_hosp_field    = 'Thirty_day_mort'
patients_cols = [patient_field,sex_field, ethnic_field, death_ind_field, death_date_field, death_date_code_field, 
                 mort_in_hosp_field]

##### Fields in table admissions ########
admission_field           = 'admission_id'
diagnosis_field           = 'episode_diagnoses'
admn_date_field           = 'admission_date_time'
admn_discharge_field      = 'discharge_date_time'
admn_date_code_field      = 'admission_date_code_time'
admn_discharge_code_field = 'discharge_date_code_time'
lengthofstay_field        = 'lengthofstay'
isPneumonia_field         = 'isPneumonia'
mortal_admin_field        = 'mortal_admin'
comorbidity_field         = 'Comorbidity_score'
icu_admin_field           = 'icu_count\r'
no_eobs_field             = 'no_obs_eobs'
no_haematology_field      = 'no_haematology_eobs'
meds_drug_field           = 'Drug_supplied'
specific_comorb_field     = 'Specific Comorbidity'
prev_admin_field          = 'had_Prev_admin'
has_spin_field            = 'adm_has_spin'
cubr65_field              = 'CURB65'
antibiot_4h_field         = 'antibiotic_4h'

 ##### Fields in table eObservations #####
eObs_time_field      = 'timestamp'
eObs_time_code_field = 'timestamp_code'
eObs_time_prev_obs   = 'time_since_prev_obs_in_mins'
eObs_resprate_field  = 'rr'
eObs_sbp_field       = 'sbp'
eObs_dbp_field       = 'dbp'
eObs_newscore_field  = 'ews'
eObs_heartrate_field = 'heart_rate'
eObs_temptr_field    = 'temperature\r'
eObs_oxygen_field    = 'Oxygen_Saturation'

##### Fields in table haematological tests #####
test_code_field      = 'local_test_code'
test_time_field      = 'sample_collection_date_time'
test_time_code_field = 'sample_collection_date_code_time'
haemt_cols = [test_code_field, test_time_field, test_time_code_field]

In [4]:
data = pickle.load( open('DataFrame_pickles/df_patients_admissions_v3.pickle', 'rb'))
df_patients   = data[0]
df_admissions = data[1]

# 1.  Meds Preprocessing
### 1.1. Standarise names of drugs

In [5]:
df_names = pd.read_csv(r'/home/d/dlr10/Documents/01_Preprocessing/input_drugs_names.csv')
df_names = dict(zip(df_names['Drug'],df_names['Final Name']))
display(df_names)
df_meds.head()

{'ABACAVIR / LAMIVUDINE': 'ABACAVIR / LAMIVUDINE',
 'ACETAZOLAMIDE [EP]': 'ACETAZOLAMIDE [EP]',
 'ACICLOVIR': 'ACICLOVIR',
 'ACLIDINIUM': 'ACLIDINIUM',
 'ACLIDINIUM+FORMOTEROL FUMARATE': 'ACLIDINIUM',
 'ADCAL': 'ADCAL',
 'Adcal': 'ADCAL',
 'ADCAL D3': 'ADCAL',
 'ADCAL D3 DISSOLVE': 'ADCAL',
 'ADCAL-D3': 'ADCAL',
 'ADCAL-D3 LEMON': 'ADCAL',
 'ADIZEM-XL': 'ADIZEM-XL',
 'AEROCHAMBER': 'AEROCHAMBER',
 'ALENDRONIC ACID': 'ALENDRONIC ACID',
 'ALFUZOSIN HYDROCHLORIDE': 'ALFUZOSIN HYDROCHLORIDE',
 'ALGICON': 'ALGICON',
 'ALLOPURINOL': 'ALLOPURINOL',
 'ALVESCO': 'ALVESCO',
 'AMILORIDE HCL': 'AMILORIDE HCL',
 'AMINOPHYLLINE': 'AMINOPHYLLINE',
 'AMIODARONE HCL': 'AMIODARONE HCL',
 'AMITRIPTYLINE': 'AMITRIPTYLINE',
 'AMLODIPINE': 'AMLODIPINE',
 'AMOXICILLIN': 'AMOXICILLIN',
 'AMOXIDENT': 'AMOXICILLIN',
 'AMOXIL': 'AMOXICILLIN',
 'AMOXYCILLIN': 'AMOXICILLIN',
 'AMOXYMED': 'AMOXICILLIN',
 'ANASTROZOLE': 'ANASTROZOLE',
 'ANGITIL SR 120': 'ANGITIL',
 'ANGITIL SR 90': 'ANGITIL',
 'APIXABAN': 'APIXABAN'

,patient_id,drug_type,drug_readcode,drug_name,formulation,strength,dose,frequency,route,duration,start_date_type,start_date,current_status,end_date_known,end_date\r
0,16378919,1.0,e31m.,AUGMENTIN,i-v inj,1.2g,1.2,9.0,6.0,2.0,3.0,2016-02-02,1.0,NaN,0000-00-00\r
1,16378919,1.0,e758.,DOXYCYCLINE,capsules,100mg,200.0,32.0,1.0,2.0,3.0,2016-02-02,1.0,NaN,0000-00-00\r
2,55739091,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,\r
3,64162874,1.0,eb93.,MEROPENEM,inj powder,1g,1.0,9.0,6.0,1.0,3.0,2016-08-07,2.0,Y,2016-08-07\r
4,64162874,1.0,eb93.,MEROPENEM,inj powder,1g,1.0,9.0,6.0,2.0,2.0,0000-00-00,1.0,NaN,0000-00-00\r


### 1.2. Filtering the raw data of meds with rows with information (either in the dosis, or the formula )

In [6]:

meds_time_test_field = 'start_date'
meds_drug_name_field = 'drug_name'
meds_drug_dose_field = 'dose'
meds_drug_strngth_field = 'strength'
meds_drug_codes_field   = 'drug_readcode'
meds_formulation_field  = 'formulation'
meds_concat_drug_form   = 'drug_form'

df_meds_new =  df_meds[((df_meds[meds_drug_name_field].notna()) & 
                        (df_meds[meds_drug_codes_field].notna()) &
                        (df_meds[meds_time_test_field].notna()) &
                        (df_meds[meds_drug_strngth_field].notna()) &
                        (df_meds[meds_formulation_field].notna()) &
                        (df_meds[meds_drug_dose_field].notna()))].copy()
df_meds_new[meds_drug_name_field]   = df_meds_new[meds_drug_name_field].apply(lambda x: df_names[x] if x in df_names.keys() else x)


df_meds_new[meds_formulation_field] = df_meds_new[meds_formulation_field].apply(lambda x:  'inj' if 'inj' in x else 'caps')
df_meds_new [meds_concat_drug_form] = df_meds_new.apply(lambda x: '-'.join([x[meds_drug_name_field],x[meds_formulation_field]]), axis = 1)
display(df_meds_new.head())

print('Medicine records before deleting', len(df_meds))
print('Filtered Medicine Records:', len(df_meds_new))
print("Portion of deleted records", '{:.2f}'.format(1- len(df_meds_new)/len(df_meds)))
print("")
print('Number of unique medicines (drug names field):', len(df_meds_new[meds_drug_name_field].unique().tolist()))

,patient_id,drug_type,drug_readcode,drug_name,formulation,strength,dose,frequency,route,duration,start_date_type,start_date,current_status,end_date_known,end_date\r,drug_form
0,16378919,1.0,e31m.,AUGMENTIN,inj,1.2g,1.2,9.0,6.0,2.0,3.0,2016-02-02,1.0,NaN,0000-00-00\r,AUGMENTIN-inj
1,16378919,1.0,e758.,DOXYCYCLINE,caps,100mg,200.0,32.0,1.0,2.0,3.0,2016-02-02,1.0,NaN,0000-00-00\r,DOXYCYCLINE-caps
3,64162874,1.0,eb93.,MEROPENEM,inj,1g,1.0,9.0,6.0,1.0,3.0,2016-08-07,2.0,Y,2016-08-07\r,MEROPENEM-inj
4,64162874,1.0,eb93.,MEROPENEM,inj,1g,1.0,9.0,6.0,2.0,2.0,0000-00-00,1.0,NaN,0000-00-00\r,MEROPENEM-inj
5,64162874,1.0,e758.,DOXYCYCLINE,caps,100mg,200.0,32.0,1.0,2.0,2.0,0000-00-00,1.0,NaN,0000-00-00\r,DOXYCYCLINE-caps


Medicine records before deleting 36406
Filtered Medicine Records: 24082
Portion of deleted records 0.34

Number of unique medicines (drug names field): 360



### 1.3. Processing and merging meds table with patients and admissions tables

In [7]:
A = df_meds_new[patient_field].unique().tolist()
B = df_patients[patient_field].unique().tolist()
C = df_admissions[patient_field].unique().tolist()
D = set(A) & set(B) & set(C)

print(len(A),'Patients in medicines')
print(len(B),'Patients in patients')
print(len(C),'Patients in admissions')
print(len(D),'Patients in all')

patients_to_process_meds = list(D)

6863 Patients in medicines
14946 Patients in patients
14946 Patients in admissions
5960 Patients in all


In [8]:
t = time.time()
k = 0; l = 0; u = 0; m1 = 0; m2 = 0
nono_pat  = []
df_admissions[meds_drug_field] = ""
for patient in patients_to_process_meds:
    admission_s = df_admissions[df_admissions[patient_field] == patient]
    medicine    = df_meds_new[df_meds_new[patient_field] == patient]
    
    if len(admission_s) == 1 and len(medicine) > 0: #patients one admission and drugs assigned
        for idx in admission_s.index.tolist():
            df_admissions.at[idx,meds_drug_field] = '_'.join(medicine[meds_concat_drug_form].unique().tolist())
        m1 = m1 + 1
    elif len(admission_s) > 1 and len(medicine) > 0: # patients with more than one admission and drugs assigned
        
        for i_adm in range(len(admission_s)):
            adm_time = admission_s.iloc[i_adm][admn_date_field]
            idx = admission_s.iloc[i_adm].name
            for i_med in range(len(medicine)):
                drg_time = medicine.iloc[i_med][meds_time_test_field]
                drg_time = [int(x) for x in drg_time.split('-')]
                if (adm_time.year == drg_time[0]) and  (adm_time.month == drg_time[1]):
                    i_med_fin = i_med + 2
                    
                    while i_med_fin < len(medicine):
                        
                        drg_time_temp = [int(x) for x in medicine.iloc[i_med_fin][meds_time_test_field].split('-')]
                        if 0 != drg_time_temp[0]: break
                        i_med_fin = i_med_fin + 1
                    
                    df_admissions.at[idx,meds_drug_field] = '_'.join(medicine.iloc[i_med:i_med_fin][meds_concat_drug_form].unique().tolist())
                    break
                    
        k = k + 1        
    elif len(admission_s) >= 1 and len(medicine) == 0:
        nono_pat.append(patient)
        u = u + 1
    else:
        l = l + 1
    
print(m1, "Number of patients with ONE admission and drug supplied")    
print(k, "Number of patients with MORE than ONE admission and drug supplied")
print(u, "Number of patients with MORE than ONE admission and NO drug supplied")
print(l, "Otherwise")

print(len(patients_to_process_meds), "patients processed")
print('Time elapsed:',time.time() - t)

4417 Number of patients with ONE admission and drug supplied
1543 Number of patients with MORE than ONE admission and drug supplied
0 Number of patients with MORE than ONE admission and NO drug supplied
0 Otherwise
5960 patients processed
Time elapsed: 21.338926315307617


In [9]:
display(df_admissions.head(5))
print(len(df_admissions), "total admissions")
print(len(df_admissions[df_admissions[meds_drug_field]!=""]), "admissions WITH drug information")
print(len(df_admissions[df_admissions[meds_drug_field]==""]), "admissions WITHOUT drug information")
print(len(df_admissions[meds_drug_field].unique().tolist()), "unique drug keys")

,admission_id,episode_diagnoses,patient_id,age_at_admission\r,admission_date_time,admission_date_code_time,discharge_date_time,discharge_date_code_time,lengthofstay,isPneumonia,mortal_admin,icu_count\r,Comorbidity_score,no_obs_eobs,no_haematology_eobs,Drug_supplied
1,7763466,"||J18.1,J84.9,C34.9,R26.8,E11.9,R15.X,R32.X,H91.9",68825097,90,2017-10-26 05:36:00,17465.233333,2017-10-26 17:59:00,17465.749306,0 days 12:23:00,1,0,0,3,10,0,
2,58250780,"||J18.1,J44.0,C34.9,I71.4,R60.0,I73.9,Z92.1,I1...",68825097,91,2017-11-06 23:01:00,17476.959028,2017-11-09 05:50:00,17479.243056,2 days 06:49:00,1,1,0,15,20,0,
4,92464775,"||J18.1,N17.9,F05.9,J90.X,R33.X,G20.X,R63.8,E1...",18544317,91,2018-03-11 18:20:00,17601.763889,2018-03-16 18:00:00,17606.750000,4 days 23:40:00,1,1,0,6,45,1,
5,74554266,"||J18.9,E87.1,A09.0,R11.X,I10.X,M10.9,C91.1,F3...",42309321,72,2018-02-19 07:58:00,17581.331944,2018-02-23 18:00:00,17585.750000,4 days 10:02:00,1,0,0,0,17,6,CO-AMOXICLAV-inj_CO-AMOXICLAV-caps
6,67812046,"||J45.9,J18.9,F17.1||J45.9,J18.9,F17.1",80081107,40,2018-01-05 13:14:00,17536.551389,2018-01-09 16:15:00,17540.677083,4 days 03:01:00,1,0,0,0,18,1,CO-AMOXICLAV-inj


18537 total admissions
6207 admissions WITH drug information
12330 admissions WITHOUT drug information
945 unique drug keys


### 1.4. Receive drug binary field in admissions

In [10]:
df_admissions['receivedMedicines'] = df_admissions[meds_drug_field].apply(lambda x : 1 if x != "" else 0)
df_admissions = df_admissions.drop(columns = [meds_drug_field])

In [11]:
display(df_admissions.head(3))



print("Admissions mortality = 0")

print(len(df_admissions[(df_admissions['mortal_admin']== 0) & (df_admissions['receivedMedicines'] == 1)]), 
     "Admissions with Madicine = 1")
print(len(df_admissions[(df_admissions['mortal_admin']== 0) & (df_admissions['receivedMedicines'] == 0)]),
      "Admissions with Madicine = 0")
print("")
print("Admissions mortality = 1")
print(len(df_admissions[(df_admissions['mortal_admin']== 1) & (df_admissions['receivedMedicines'] == 1)]),
      "Admissions with Madicine = 1")
print(len(df_admissions[(df_admissions['mortal_admin']== 1) & (df_admissions['receivedMedicines'] == 0)]),
      "Admissions with Madicine = 0")

,admission_id,episode_diagnoses,patient_id,age_at_admission\r,admission_date_time,admission_date_code_time,discharge_date_time,discharge_date_code_time,lengthofstay,isPneumonia,mortal_admin,icu_count\r,Comorbidity_score,no_obs_eobs,no_haematology_eobs,receivedMedicines
1,7763466,"||J18.1,J84.9,C34.9,R26.8,E11.9,R15.X,R32.X,H91.9",68825097,90,2017-10-26 05:36:00,17465.233333,2017-10-26 17:59:00,17465.749306,0 days 12:23:00,1,0,0,3,10,0,0
2,58250780,"||J18.1,J44.0,C34.9,I71.4,R60.0,I73.9,Z92.1,I1...",68825097,91,2017-11-06 23:01:00,17476.959028,2017-11-09 05:50:00,17479.243056,2 days 06:49:00,1,1,0,15,20,0,0
4,92464775,"||J18.1,N17.9,F05.9,J90.X,R33.X,G20.X,R63.8,E1...",18544317,91,2018-03-11 18:20:00,17601.763889,2018-03-16 18:00:00,17606.750000,4 days 23:40:00,1,1,0,6,45,1,0


Admissions mortality = 0
5458 Admissions with Madicine = 1
9232 Admissions with Madicine = 0

Admissions mortality = 1
749 Admissions with Madicine = 1
3098 Admissions with Madicine = 0


# 2. Specific CAP Comorbidities

In [12]:
ICD_codes_CAP = 'A01.0, A02.2, A20.2, A21.2, A22.1, A31.0, A37.0, A37.1, A43.0, A48.1, B01.2, B05.2, B37.1, B38.0,' \
             'B38.1, B38.2, B39.0, B39.1, B39.2, B58.3, B59, B77.8, J12.0, J12.1, J12.2, J12.3, J12.8, J12.8,'\
             'J12.9, J13, J14, J15.0, J15.1, J15.2, J15.3, J15.4, J15.5, J15.6, J15.7, J15.8, J15.9,J16.0, J16.8, J17.0,'\
             'J17.1, J17.2, J17.3, J17.8, J18.0, J18.1, J18.8, J18.9, J85.1, J85.2, B59'
ICD_codes_CAP = re.sub(r"\s+", "", ICD_codes_CAP)
ICD_codes_CAP = ICD_codes_CAP.split(',')
specific_comorbiditiy_codes = ['A15.x - A19.x', 'A30.x - A99.x', 'B00.x - B09.x', 
                               'B35.x - B49.x', 'I26.x - I28.x', 'I50.x']

### 2.1. Adding field of specific comorbidity for respiratory science

In [13]:
t = time.time()
def admission_specific_comorbidity(patient_codes, specific_comorbiditiy_ICD_codes):
    patient_codes = re.sub(r'(^\|\|)','',patient_codes)
    patient_codes = re.sub(r'(\|\|$)','',patient_codes)
    patient_codes = re.sub(r'(,$)','',patient_codes)
    patient_codes = re.sub(r'(\|\|)',',',patient_codes)
    patient_codes = re.sub(r'(,,)',',',patient_codes)
    patient_codes = re.split(',', patient_codes)

    specific_comorbidities = False

    for specific_code in specific_comorbiditiy_ICD_codes:
        for code in patient_codes:
            if len(code) <= 3: continue
            code_div = [code.split('.')[0][0],  int(code.split('.')[0][1:])]
            if ' - ' in specific_code:
                rng = [sub_code.split('.')[0] for sub_code in specific_code.split(' - ')]
                rng = [rng[0][0], [int(rng[0][1:]), int(rng[1][1:])]]    
            else:
                
                rng = [specific_code.split('.')[0][0], [int(specific_code.split('.')[0][1:])]]

            if rng[0] != code_div[0]: continue

            if len(rng[1]) == 1: 
                if rng[1][0] == code_div[1]: 
                    specific_comorbidities = True
            else:
                if (rng[1][0] <= code_div[1]) and (code_div[1] <= rng[1][1]): 
                    specific_comorbidities = True
        specific_comorbidities = 1 if specific_comorbidities else 0
    return specific_comorbidities    

df_admissions[specific_comorb_field] = df_admissions[diagnosis_field].apply(lambda x: admission_specific_comorbidity(x, specific_comorbiditiy_codes))   
    
display(df_admissions.head(3))
print("time elapsed", time.time()-t)

,admission_id,episode_diagnoses,patient_id,age_at_admission\r,admission_date_time,admission_date_code_time,discharge_date_time,discharge_date_code_time,lengthofstay,isPneumonia,mortal_admin,icu_count\r,Comorbidity_score,no_obs_eobs,no_haematology_eobs,receivedMedicines,Specific Comorbidity
1,7763466,"||J18.1,J84.9,C34.9,R26.8,E11.9,R15.X,R32.X,H91.9",68825097,90,2017-10-26 05:36:00,17465.233333,2017-10-26 17:59:00,17465.749306,0 days 12:23:00,1,0,0,3,10,0,0,0
2,58250780,"||J18.1,J44.0,C34.9,I71.4,R60.0,I73.9,Z92.1,I1...",68825097,91,2017-11-06 23:01:00,17476.959028,2017-11-09 05:50:00,17479.243056,2 days 06:49:00,1,1,0,15,20,0,0,0
4,92464775,"||J18.1,N17.9,F05.9,J90.X,R33.X,G20.X,R63.8,E1...",18544317,91,2018-03-11 18:20:00,17601.763889,2018-03-16 18:00:00,17606.750000,4 days 23:40:00,1,1,0,6,45,1,0,0


time elapsed 5.463075160980225


In [14]:
df_admissions['episode_diagnoses'].tolist()[0]

'||J18.1,J84.9,C34.9,R26.8,E11.9,R15.X,R32.X,H91.9'

In [15]:
print("Number of admissions WITHOUT specific comorbidities ")
#print(len(df_admissions[df_admissions[specific_comorb_field] ==False]))
print(len(df_admissions[(df_admissions['mortal_admin']== 0) & (df_admissions[specific_comorb_field] ==False)]),
      "Admissions mortality = 0")
print(len(df_admissions[(df_admissions['mortal_admin']== 1) & (df_admissions[specific_comorb_field] ==False)]),
      "Admissions mortality = 1")
print("")
print("Number of admissions WITH specific comorbidities ")
#print(len(df_admissions[df_admissions[specific_comorb_field] ==True]))

print(len(df_admissions[(df_admissions['mortal_admin']== 0) & (df_admissions[specific_comorb_field] ==True)]),
      "Admissions mortality = 0")
print(len(df_admissions[(df_admissions['mortal_admin']== 1) & (df_admissions[specific_comorb_field] ==True)]),
      "Admissions mortality = 1")

df_admissions[specific_comorb_field] = df_admissions[specific_comorb_field].apply(lambda x: 1 if x == True else 0)
display(df_admissions.head(3))

Number of admissions WITHOUT specific comorbidities 
8438 Admissions mortality = 0
1317 Admissions mortality = 1

Number of admissions WITH specific comorbidities 
6252 Admissions mortality = 0
2530 Admissions mortality = 1


,admission_id,episode_diagnoses,patient_id,age_at_admission\r,admission_date_time,admission_date_code_time,discharge_date_time,discharge_date_code_time,lengthofstay,isPneumonia,mortal_admin,icu_count\r,Comorbidity_score,no_obs_eobs,no_haematology_eobs,receivedMedicines,Specific Comorbidity
1,7763466,"||J18.1,J84.9,C34.9,R26.8,E11.9,R15.X,R32.X,H91.9",68825097,90,2017-10-26 05:36:00,17465.233333,2017-10-26 17:59:00,17465.749306,0 days 12:23:00,1,0,0,3,10,0,0,0
2,58250780,"||J18.1,J44.0,C34.9,I71.4,R60.0,I73.9,Z92.1,I1...",68825097,91,2017-11-06 23:01:00,17476.959028,2017-11-09 05:50:00,17479.243056,2 days 06:49:00,1,1,0,15,20,0,0,0
4,92464775,"||J18.1,N17.9,F05.9,J90.X,R33.X,G20.X,R63.8,E1...",18544317,91,2018-03-11 18:20:00,17601.763889,2018-03-16 18:00:00,17606.750000,4 days 23:40:00,1,1,0,6,45,1,0,0


# 3. Previous Admissions boolean variable

In [16]:
df_prev.head(3)

,episode_diagnoses,patient_id,prev_admission_id,admission_date,admission_time,discharge_date,discharge_time\r
0,"||H26.9,I10.X,E11.9",68825097,7763466,2007-04-25 00:00:00,1200,2007-04-26 00:00:00,1454
1,"||H26.9,I10.X,E11.9",68825097,3305648,2007-04-25 00:00:00,1200,2007-04-26 00:00:00,1454
2,"||H26.9,I10.X,E11.9",68825097,58250780,2007-04-25 00:00:00,1200,2007-04-26 00:00:00,1454


### 3.1. Pre-processing df_prev

In [17]:
# Processing previous admissions table times
df_prev = df_prev.rename(columns = {'discharge_time\r':  'discharge_time' })
df_prev['admission_date']          = pd.to_datetime(df_prev['admission_date'], format='%Y-%m-%d %H:%M:%S')
df_prev['admission_time']          = df_prev['admission_time'].apply(lambda x: datetime.timedelta(hours = int(x/100), minutes = x%100))
df_prev[admn_date_field]           = df_prev['admission_date'] + df_prev['admission_time']
df_prev['discharge_date']          = pd.to_datetime(df_prev['discharge_date'], format='%Y-%m-%d %H:%M:%S')
df_prev['discharge_time']          = df_prev['discharge_time'].apply(lambda x: datetime.timedelta(hours = int(x/100), minutes = x%100))
df_prev[admn_discharge_field]      = df_prev['discharge_date'] + df_prev['discharge_time']
df_prev = df_prev.drop(columns = ['admission_date', 'admission_time','discharge_date','discharge_time'])
df_prev.head(3)

,episode_diagnoses,patient_id,prev_admission_id,admission_date_time,discharge_date_time
0,"||H26.9,I10.X,E11.9",68825097,7763466,2007-04-25 12:00:00,2007-04-26 14:54:00
1,"||H26.9,I10.X,E11.9",68825097,3305648,2007-04-25 12:00:00,2007-04-26 14:54:00
2,"||H26.9,I10.X,E11.9",68825097,58250780,2007-04-25 12:00:00,2007-04-26 14:54:00


In [18]:
A = df_prev[patient_field].unique().tolist()
B = df_patients[patient_field].unique().tolist()
C = df_admissions[patient_field].unique().tolist()
D = set(A) & set(B) & set(C)

print(len(A),'Patients in previous admissions')
print(len(B),'Patients in patients')
print(len(C),'Patients in admissions')
print(len(D),'Patients in all')

16082 Patients in previous admissions
14946 Patients in patients
14946 Patients in admissions
13373 Patients in all


In [19]:
t = time.time()
df_admissions[prev_admin_field] = np.nan
for idx, admin_info in df_admissions.iterrows(): 
    
    adm_patient  = admin_info[patient_field]
    adm_date     = admin_info[admn_date_field]
    adm_prev_min = adm_date - relativedelta(months=12)
    
    adm_after_2016 = len(df_admissions[(df_admissions[patient_field]==adm_patient) & 
                        (adm_prev_min <= df_admissions[admn_date_field]) &
                        (df_admissions[admn_date_field] < adm_date)])
    
    adm_before_2016 = len(df_prev[(df_prev[patient_field]==adm_patient) &
                         (adm_prev_min <= df_prev[admn_date_field]) &
                         (df_prev[admn_date_field] < adm_date)])
    
    df_admissions.at[idx, prev_admin_field]  = 1 if (adm_after_2016 > 0) or (adm_before_2016 > 0) else 0

print("time elapsed:", time.time() - t)

time elapsed: 98.59991645812988


In [20]:
print("Number of admissions WITHOUT previous admissions")
#print(len(df_admissions[df_admissions[specific_comorb_field] ==False]))
print(len(df_admissions[(df_admissions['mortal_admin']== 0) & (df_admissions[prev_admin_field] == 0)]),
      "Admissions mortality = 0")
print(len(df_admissions[(df_admissions['mortal_admin']== 1) & (df_admissions[prev_admin_field] == 0)]),
      "Admissions mortality = 1")
print("")
print("Number of admissions WITH previous admissions")
#print(len(df_admissions[df_admissions[specific_comorb_field] ==True]))

print(len(df_admissions[(df_admissions['mortal_admin']== 0) & (df_admissions[prev_admin_field] == 1)]),
      "Admissions mortality = 0")
print(len(df_admissions[(df_admissions['mortal_admin']== 1) & (df_admissions[prev_admin_field] == 1)]),
      "Admissions mortality = 1")

display(df_admissions.head(3))

Number of admissions WITHOUT previous admissions
6215 Admissions mortality = 0
1258 Admissions mortality = 1

Number of admissions WITH previous admissions
8475 Admissions mortality = 0
2589 Admissions mortality = 1


,admission_id,episode_diagnoses,patient_id,age_at_admission\r,admission_date_time,admission_date_code_time,discharge_date_time,discharge_date_code_time,lengthofstay,isPneumonia,mortal_admin,icu_count\r,Comorbidity_score,no_obs_eobs,no_haematology_eobs,receivedMedicines,Specific Comorbidity,had_Prev_admin
1,7763466,"||J18.1,J84.9,C34.9,R26.8,E11.9,R15.X,R32.X,H91.9",68825097,90,2017-10-26 05:36:00,17465.233333,2017-10-26 17:59:00,17465.749306,0 days 12:23:00,1,0,0,3,10,0,0,0,0.0
2,58250780,"||J18.1,J44.0,C34.9,I71.4,R60.0,I73.9,Z92.1,I1...",68825097,91,2017-11-06 23:01:00,17476.959028,2017-11-09 05:50:00,17479.243056,2 days 06:49:00,1,1,0,15,20,0,0,0,1.0
4,92464775,"||J18.1,N17.9,F05.9,J90.X,R33.X,G20.X,R63.8,E1...",18544317,91,2018-03-11 18:20:00,17601.763889,2018-03-16 18:00:00,17606.750000,4 days 23:40:00,1,1,0,6,45,1,0,0,1.0


# 4. Fields of Curb65 and Spin records

In [21]:
df_spin['date_time_arrived'] = pd.to_datetime(df_spin['date_time_arrived'], format='%Y-%m-%d %H:%M:%S')
print("Lenght of spin table", len(df_spin))
df_spin = df_spin.drop(df_spin[df_spin['curb65_score'].isna()].index, axis =0)
print(len(df_spin))
display(df_spin.head(2))

Lenght of spin table 6812
5891


,patient_id,date_time_arrived,cxr_4h,cxr_cap,cxr_date,cxr_time,o2_1h,o2_prescribed_1h,curb65_recorded,curb65_score,ab_4h,reason_ab_not_concordant,ab_severity,ab_date_time\r
0,22377059,2016-01-05 12:27:00,Y,Y,2015-01-05,15:35,Y,N,Y,2.0,Y,NaN,N,2016-01-05 12:50:00\r
1,83768183,2016-01-06 13:00:00,Y,Y,2016-01-06,14:53,Y,N,Y,0.0,Y,NaN,Y,2016-01-06 16:00:00\r


In [22]:
A = df_spin[patient_field].unique().tolist()
B = df_patients[patient_field].unique().tolist()
C = df_admissions[patient_field].unique().tolist()
D = set(A) & set(B) & set(C)

print(len(A),'Patients in spin table')
print(len(B),'Patients in patients')
print(len(C),'Patients in admissions')
print(len(D),'Patients in all')

patient_to_process = list(D)

5348 Patients in spin table
14946 Patients in patients
14946 Patients in admissions
4638 Patients in all


In [23]:
t = time.time()
h = 0; j=0

df_admissions[has_spin_field]    = np.nan
df_admissions[cubr65_field]      = np.nan
df_admissions[antibiot_4h_field] = np.nan

for patient in patient_to_process:
    pat_spns = df_spin[df_spin[patient_field] == patient]
    pat_adms = df_admissions[df_admissions[patient_field] == patient]    
    
    for i_adm in range(len(pat_adms)):
        adm_time = pat_adms.iloc[i_adm][admn_date_field]
        max_time = adm_time + relativedelta(hours=12)
        min_time = adm_time - relativedelta(hours=12)
        idx      = pat_adms.iloc[i_adm].name
        
        adm_spin = pat_spns[(min_time <= pat_spns['date_time_arrived']) & (pat_spns['date_time_arrived'] <= max_time)]
        # There are 4 admissions that coincide with two spin records, that are almost similar, so takes the first record
        
        if len(adm_spin) > 0:
            df_admissions.at[idx, has_spin_field]    = 1
            df_admissions.at[idx, cubr65_field]      = adm_spin.iloc[0]['curb65_score']
            df_admissions.at[idx, antibiot_4h_field] = 1 if adm_spin.iloc[0]['ab_4h'] == "Y" else 0
        
print("elapsed", time.time()-t) 

elapsed 23.325876235961914


In [24]:
display(df_admissions.head(2))

,admission_id,episode_diagnoses,patient_id,age_at_admission\r,admission_date_time,admission_date_code_time,discharge_date_time,discharge_date_code_time,lengthofstay,isPneumonia,...,icu_count\r,Comorbidity_score,no_obs_eobs,no_haematology_eobs,receivedMedicines,Specific Comorbidity,had_Prev_admin,adm_has_spin,CURB65,antibiotic_4h
1,7763466,"||J18.1,J84.9,C34.9,R26.8,E11.9,R15.X,R32.X,H91.9",68825097,90,2017-10-26 05:36:00,17465.233333,2017-10-26 17:59:00,17465.749306,0 days 12:23:00,1,...,0,3,10,0,0,0,0.0,NaN,NaN,NaN
2,58250780,"||J18.1,J44.0,C34.9,I71.4,R60.0,I73.9,Z92.1,I1...",68825097,91,2017-11-06 23:01:00,17476.959028,2017-11-09 05:50:00,17479.243056,2 days 06:49:00,1,...,0,15,20,0,0,0,1.0,NaN,NaN,NaN


In [25]:
df_admissions.isnull().sum()

admission_id                    0
episode_diagnoses               0
patient_id                      0
age_at_admission\r              0
admission_date_time             0
admission_date_code_time        0
discharge_date_time             0
discharge_date_code_time        0
lengthofstay                    0
isPneumonia                     0
mortal_admin                    0
icu_count\r                     0
Comorbidity_score               0
no_obs_eobs                     0
no_haematology_eobs             0
receivedMedicines               0
Specific Comorbidity            0
had_Prev_admin                  0
adm_has_spin                14020
CURB65                      14020
antibiotic_4h               14020
dtype: int64

In [26]:
pickling_data = [df_patients, df_admissions]
pickle.dump(pickling_data, open('DataFrame_pickles/df_patients_admissions_2016_18_v4.pickle', 'wb'))